### SETUP INICIAL DO PROJETO

In [235]:

#importação das bibliotecase pacotes necessários para a análise

import json
import numpy as np
import os
import pandas as pd
import pandas_gbq as gbq
import re
import seaborn as sns
from dotenv import load_dotenv
from google.cloud import bigquery
from google.cloud.bigquery_storage import BigQueryReadClient
from google.oauth2 import service_account

# Carrega o .env: onde estão as credenciais do projeto/repositório
load_dotenv("/mnt/c/Users/wrpen/OneDrive/Desktop/df_lh/.env")

# Detectar ambiente: como eu estou usando wsl-ubuntu, no VS Code  -  Windows, estava dando conflitos de path
if os.name == "nt":  # se Windows
    credentials_path = r"C:\Temp\desafiolh-445818-3cb0f62cb9ef.json"
else:  # se WSL/Linux
    credentials_path = "/mnt/c/Temp/desafiolh-445818-3cb0f62cb9ef.json"

# Parâmetros injetados pelo Papermill ou definidos manualmente, caso não existam no ambiente
# Tables_to_process: lista de tabelas que serão processadas
# Output_dataset: nome do dataset onde os dados processados serão armazenados, neste caso, raw_data_cleaned
if 'tables_to_process' not in locals():
    tables_to_process = [
        "desafioadventureworks-446600.raw_data.humanresources_employee"       
    ]

if 'output_dataset' not in locals():
    output_dataset = "desafioadventureworks-446600.raw_data_cleaned"

# Configs do cliente BigQuery: input de project e location de acordo com dados no Bigquery
credentials = service_account.Credentials.from_service_account_file(credentials_path)
client = bigquery.Client(credentials=credentials, project=os.getenv("BIGQUERY_PROJECT"), location="us-central1")

# Print de verificação das credenciais do BigQuery e seu path
print("Credenciais do BigQuery:", os.getenv("GOOGLE_APPLICATION_CREDENTIALS"))


Credenciais do BigQuery: /mnt/c/Temp/desafiolh-445818-3cb0f62cb9ef.json


In [236]:
# Print com a tabela que vai ser processada nesse notebook

print("Tabelas a processar:", tables_to_process)

Tabelas a processar: ['desafioadventureworks-446600.raw_data.humanresources_employee']


In [237]:
# Nome do dataset no Bigquery com os dados brutos extraídos pelo Meltano

dataset_id = 'raw_data'
print(dataset_id)


raw_data


In [238]:
# Lista de tabelas do dataset raw_data no Bigquery

tables = client.list_tables('raw_data')
print("Tabelas disponíveis:")
for table in tables:
    print(table.table_id)

Tabelas disponíveis:
humanresources_employee
person_address
person_businessentity
person_person
production_location
production_product
production_productcategory
production_productinventory
production_productsubcategory
sales_customer
sales_salesorderdetail
sales_salesorderheader
sales_salesterritory
sales_store


### Exploratory Data Analysis (EDA) e Data Cleaning

Glossário dos dados:

df = dataframe
bkp = backup (cópia)

In [239]:
# Configuração para que o df exiba todas as colunas e todas as linhas completas, e também, 
# exiba o formato numérico com 2 dígitos após a vírgula

pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', 1000)
pd.options.display.float_format = '{:.2f}'.format

In [240]:
# Inicializar o cliente do BigQuery Storage
bqstorage_client = BigQueryReadClient(credentials=credentials)

# Dicionário para armazenar df processados, permitindo organizar e acessar resultados intermediários ou finais por chaves - (nomes ou identificadores)
df_processados = {}

# Iteraração das tabelas e armazenamento em df
for input_table in tables_to_process:
    print(f"Processando tabela: {input_table}")
    
    # Nome da tabela
    table_name = input_table.split(".")[-1]  # Extrai o nome da tabela
    
    # Etapa 1: lê os dados da tabela do BigQuery com pyarrow
    print("Lendo os dados do BigQuery...")
    query = f"SELECT * FROM `{input_table}`"
    EDA_humanresources_employee_raw = client.query(query).to_dataframe(bqstorage_client=bqstorage_client)

    # Etapa 2: transforma JSON em formato tabular
    print("Transformando os dados para formato tabular...")

    # Verifica se há colunas com dados em formato JSON
    if EDA_humanresources_employee_raw.shape[1] == 1 and isinstance(EDA_humanresources_employee_raw.iloc[0, 0], str):
        try:
            print("Normalizando dados JSON...")

            # Substitui `null` por `None` e carrega o JSON: BigQuery utiliza 'null' para valores ausentes em JSON, mas o pandas utiliza None ou NaN
            EDA_humanresources_employee = pd.json_normalize(
                EDA_humanresources_employee_raw.iloc[:, 0].apply(lambda x: json.loads(x.replace("null", "None")))
            )
        except Exception as e:
            print(f"Erro ao normalizar JSON: {e}")
            EDA_humanresources_employee = EDA_humanresources_employee  # Caso falhar, mantém os dados brutos apra efeito de análise
    else:
        EDA_humanresources_employee = EDA_humanresources_employee_raw

    # Armazenar o df limpo no dicionário para armazenar df processados
    df_processados[table_name] = EDA_humanresources_employee
    print(f"Tabela {table_name} processada e armazenada com sucesso.")

# Print de validação
print("Todas as tabelas foram processadas com sucesso!")


Processando tabela: desafioadventureworks-446600.raw_data.humanresources_employee
Lendo os dados do BigQuery...
Transformando os dados para formato tabular...
Tabela humanresources_employee processada e armazenada com sucesso.
Todas as tabelas foram processadas com sucesso!


In [241]:
#Exibe as primeiras linhas do df
print(EDA_humanresources_employee_raw.head())


                                                                                                                                                                                                                                                                                                                                                                                                                 data                _sdc_extracted_at                 _sdc_received_at                  _sdc_batched_at _sdc_deleted_at  _sdc_sequence  _sdc_table_version
0           {"birthdate":"1969-01-29","businessentityid":1,"currentflag":true,"gender":"M","hiredate":"2009-01-14","jobtitle":"Chief Executive Officer","loginid":"adventure-works\\ken0","maritalstatus":"S","modifieddate":"2014-06-30T00:00:00","nationalidnumber":"295847284","organizationnode":"/","rowguid":"f01251e5-96a3-448d-981e-0f99d789110d","salariedflag":true,"sickleavehours":69,"vacationhours":99} 2025-01-04 15:00:48.630217+00:00 2025-

In [242]:
# Substituir `null` por `None`, carregar o JSON e retornar `None` em caso de erro, exibindo uma mensagem com o valor problemático.

def clean_and_load_json(value):
    """Função para corrigir e carregar JSON."""
    try:
        value = value.replace("null", "None")
        return json.loads(value)
    except Exception as e:
        print(f"Erro ao processar JSON: {e}, valor problemático: {value}")
        return None  

# Normalizar os dados JSON
print("Normalizando os dados JSON...")
try:
    EDA_humanresources_employee = pd.json_normalize(
        EDA_humanresources_employee_raw.iloc[:, 0].apply(clean_and_load_json)
    )
    print("Dados normalizados com sucesso!")

    # Atribuir tipos às colunas de acordo com o banco de dados
    EDA_humanresources_employee['businessentityid'] = EDA_humanresources_employee['businessentityid'].astype('int64', errors='ignore')
    EDA_humanresources_employee['nationalidnumber'] = EDA_humanresources_employee['nationalidnumber'].astype('int64', errors='ignore')
    EDA_humanresources_employee['loginid'] = EDA_humanresources_employee['loginid'].astype('int64', errors='ignore')
    EDA_humanresources_employee['jobtitle'] = EDA_humanresources_employee['jobtitle'].astype('str', errors='ignore')
    EDA_humanresources_employee['birthdate'] = pd.to_datetime(EDA_humanresources_employee['birthdate'], errors='coerce').dt.date
    EDA_humanresources_employee['maritalstatus'] = EDA_humanresources_employee['maritalstatus'].astype('str', errors='ignore')
    EDA_humanresources_employee['gender'] = EDA_humanresources_employee['gender'].astype('str', errors='ignore')
    EDA_humanresources_employee['hiredate'] = pd.to_datetime(EDA_humanresources_employee['hiredate'], errors='coerce').dt.date
    EDA_humanresources_employee['salariedflag'] = EDA_humanresources_employee['salariedflag'].astype('bool')
    EDA_humanresources_employee['vacationhours'] = EDA_humanresources_employee['vacationhours'].astype('int64', errors='ignore')
    EDA_humanresources_employee['sickleavehours'] = EDA_humanresources_employee['sickleavehours'].astype('int64', errors='ignore')
    EDA_humanresources_employee['currentflag'] = EDA_humanresources_employee['currentflag'].astype('bool')
    EDA_humanresources_employee['rowguid'] = EDA_humanresources_employee['rowguid'].astype('str', errors='ignore')
    EDA_humanresources_employee['modifieddate'] = pd.to_datetime(EDA_humanresources_employee['modifieddate'], errors='coerce').dt.date
    EDA_humanresources_employee['organizationnode'] = EDA_humanresources_employee['organizationnode'].astype('str', errors='ignore')

    print("Tipos atribuídos com sucesso!")
except Exception as e:
    print(f"Erro ao normalizar os dados JSON: {e}")
    EDA_humanresources_employee = EDA_humanresources_employee_raw 

print(f"Tabela processada: {input_table}")

print(EDA_humanresources_employee.info())

Normalizando os dados JSON...
Dados normalizados com sucesso!
Tipos atribuídos com sucesso!
Tabela processada: desafioadventureworks-446600.raw_data.humanresources_employee
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 580 entries, 0 to 579
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   birthdate         580 non-null    object
 1   businessentityid  580 non-null    int64 
 2   currentflag       580 non-null    bool  
 3   gender            580 non-null    object
 4   hiredate          580 non-null    object
 5   jobtitle          580 non-null    object
 6   loginid           580 non-null    object
 7   maritalstatus     580 non-null    object
 8   modifieddate      574 non-null    object
 9   nationalidnumber  580 non-null    int64 
 10  organizationnode  580 non-null    object
 11  rowguid           580 non-null    object
 12  salariedflag      580 non-null    bool  
 13  sickleavehours    580 non-n

In [243]:
#Verificar as dimensões do df antes de verificar as duplicatas / ausentes etc

print(f"Colunas: {EDA_humanresources_employee.shape[1]}\nLinhas: {EDA_humanresources_employee.shape[0]}")

Colunas: 15
Linhas: 580


In [244]:
# Identificar duplicatas com base em 'businessentityid'
duplicatas = EDA_humanresources_employee[
    EDA_humanresources_employee.duplicated(subset=['businessentityid'], keep=False)
]

# Verificar se existem duplicatas
if not duplicatas.empty:
    # Ordenar duplicatas por 'businessentityid' e 'modifieddate'
    duplicatas_ordenadas = duplicatas.sort_values(by=['businessentityid', 'modifieddate'])

    # Exibir duplicatas ordenadas
    print("Duplicatas ordenadas:")
    print(duplicatas_ordenadas)
else:
    print("Não foram encontradas duplicatas.")

Duplicatas ordenadas:
      birthdate  businessentityid  currentflag gender    hiredate                       jobtitle                   loginid maritalstatus modifieddate  nationalidnumber organizationnode                               rowguid  salariedflag  sickleavehours  vacationhours
0    1969-01-29                 1         True      M  2009-01-14        Chief Executive Officer      adventure-works\ken0             S   2014-06-30         295847284                /  f01251e5-96a3-448d-981e-0f99d789110d          True              69             99
290  1969-01-29                 1         True      M  2009-01-14        Chief Executive Officer      adventure-works\ken0             S   2014-06-30         295847284                /  f01251e5-96a3-448d-981e-0f99d789110d          True              69             99
1    1971-08-01                 2         True      F  2008-01-31  Vice President of Engineering    adventure-works\terri0             S   2014-06-30         245797967       

In [245]:
# Remover duplicatas mantendo a última ocorrência com base em 'modifieddate', pois ela que indica a data da última modificação nos dados
# Importante, pois se houver erro na ingestão (duplicação), mantém os dados integros.

EDA_humanresources_employee = EDA_humanresources_employee.drop_duplicates(subset=['businessentityid'], keep='last')

print(f"Linhas após remover duplicatas (baseando-se na última 'modifieddate'): {len(EDA_humanresources_employee)}")

#bkp dos dados brutos
raw_data_bkp_2_sem_duplicatas = EDA_humanresources_employee.copy()


Linhas após remover duplicatas (baseando-se na última 'modifieddate'): 290


In [246]:
# Ordenar e exibir o df por 'businessentityid' e 'modifieddate'
EDA_humanresources_employee = EDA_humanresources_employee.sort_values(by=['businessentityid'])

print(EDA_humanresources_employee)

      birthdate  businessentityid  currentflag gender    hiredate                       jobtitle                   loginid maritalstatus modifieddate  nationalidnumber organizationnode                               rowguid  salariedflag  sickleavehours  vacationhours
290  1969-01-29                 1         True      M  2009-01-14        Chief Executive Officer      adventure-works\ken0             S   2014-06-30         295847284                /  f01251e5-96a3-448d-981e-0f99d789110d          True              69             99
291  1971-08-01                 2         True      F  2008-01-31  Vice President of Engineering    adventure-works\terri0             S   2014-06-30         245797967              /1/  45e8f437-670d-4409-93cb-f9424a40d6ee          True              20              1
292  1974-11-12                 3         True      M  2007-11-11            Engineering Manager  adventure-works\roberto0             M   2014-06-30         509647174            /1/1/  9bbbfb2c-e

In [247]:
#Verificar se as colunas de datas têm valores nulos (NaN em pandas), pois não pode object, para afacilitar manipulação dos dados
print(EDA_humanresources_employee.info())

<class 'pandas.core.frame.DataFrame'>
Index: 290 entries, 290 to 579
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   birthdate         290 non-null    object
 1   businessentityid  290 non-null    int64 
 2   currentflag       290 non-null    bool  
 3   gender            290 non-null    object
 4   hiredate          290 non-null    object
 5   jobtitle          290 non-null    object
 6   loginid           290 non-null    object
 7   maritalstatus     290 non-null    object
 8   modifieddate      287 non-null    object
 9   nationalidnumber  290 non-null    int64 
 10  organizationnode  290 non-null    object
 11  rowguid           290 non-null    object
 12  salariedflag      290 non-null    bool  
 13  sickleavehours    290 non-null    int64 
 14  vacationhours     290 non-null    int64 
dtypes: bool(2), int64(4), object(9)
memory usage: 32.3+ KB
None


In [248]:
# Iterar por todas as colunas do df, para verificar valores ausentes

 # Verificar valores ausentes na coluna
for column in EDA_humanresources_employee.columns:   
    missing_rows = EDA_humanresources_employee[EDA_humanresources_employee[column].isnull()]
    print(f"Coluna '{column}': {missing_rows.shape[0]} linhas ausentes.")
    
# Mostrar as primeiras linhas ausentes, se preciso for, limitar o head() para dar menos outputs ou limitar os outputs
    if not missing_rows.empty:
        print(f"Exibindo as primeiras linhas com valores ausentes em '{column}':")
        print(missing_rows.head(), "\n")
    else:
        print(f"Nenhuma linha com valores ausentes em '{column}'.\n")

Coluna 'birthdate': 0 linhas ausentes.
Nenhuma linha com valores ausentes em 'birthdate'.

Coluna 'businessentityid': 0 linhas ausentes.
Nenhuma linha com valores ausentes em 'businessentityid'.

Coluna 'currentflag': 0 linhas ausentes.
Nenhuma linha com valores ausentes em 'currentflag'.

Coluna 'gender': 0 linhas ausentes.
Nenhuma linha com valores ausentes em 'gender'.

Coluna 'hiredate': 0 linhas ausentes.
Nenhuma linha com valores ausentes em 'hiredate'.

Coluna 'jobtitle': 0 linhas ausentes.
Nenhuma linha com valores ausentes em 'jobtitle'.

Coluna 'loginid': 0 linhas ausentes.
Nenhuma linha com valores ausentes em 'loginid'.

Coluna 'maritalstatus': 0 linhas ausentes.
Nenhuma linha com valores ausentes em 'maritalstatus'.

Coluna 'modifieddate': 3 linhas ausentes.
Exibindo as primeiras linhas com valores ausentes em 'modifieddate':
      birthdate  businessentityid  currentflag gender    hiredate                    jobtitle                  loginid maritalstatus modifieddate  na

In [249]:
# Preencher 'modifieddate' ausente ou igual a 'hiredate', por ser a última data de modificação no sistema.
EDA_humanresources_employee.loc[EDA_humanresources_employee['modifieddate'].isnull() | (EDA_humanresources_employee['modifieddate'] == pd.Timestamp('1900-01-01')), 'modifieddate'] = EDA_humanresources_employee['hiredate']

# Exibir as linhas ajustadas
print("Linhas onde 'modifieddate' foi ajustado para 'hiredate':")
print(EDA_humanresources_employee.loc[EDA_humanresources_employee['modifieddate'] == EDA_humanresources_employee['hiredate']])

# bkp do df
ajustes_modifieddate = EDA_humanresources_employee.copy()

Linhas onde 'modifieddate' foi ajustado para 'hiredate':
      birthdate  businessentityid  currentflag gender    hiredate                    jobtitle                  loginid maritalstatus modifieddate  nationalidnumber organizationnode                               rowguid  salariedflag  sickleavehours  vacationhours
315  1982-11-03                26         True      M  2008-12-01  Production Control Manager   adventure-works\peter0             M   2008-12-01         277173473            /3/1/  69d5d162-e817-45e7-9dec-5d9b8310e7b1          True              41             43
500  1977-10-26               211         True      M  2009-02-28   Quality Assurance Manager   adventure-works\hazem0             S   2009-02-28         398223854            /3/2/  05c84608-f445-4f9d-bb5c-0828c309c29d          True              60             80
511  1968-09-17               222         True      M  2008-12-12            Master Scheduler  adventure-works\ascott0             S   2008-12-12      

In [250]:
# Valores únicos por coluna, para verificar se colunas como flags, normalmente booleanas, possuem apenas 1 ou 2 valores.

valores_unicos = EDA_humanresources_employee.nunique(dropna=False)

print("Valores únicos incluindo NaN:")
print(valores_unicos)

Valores únicos incluindo NaN:
birthdate           275
businessentityid    290
currentflag           1
gender                2
hiredate            164
jobtitle             67
loginid             290
maritalstatus         2
modifieddate          4
nationalidnumber    290
organizationnode    290
rowguid             290
salariedflag          2
sickleavehours       51
vacationhours       100
dtype: int64


In [251]:
# dropar colunas vazias

In [252]:
# Padronizar textos em title (ex: Production Supervisor) ou upper (ex:M)
EDA_humanresources_employee['jobtitle'] = EDA_humanresources_employee['jobtitle'].str.strip().str.title()
EDA_humanresources_employee['gender'] = EDA_humanresources_employee['gender'].str.strip().str.upper()
EDA_humanresources_employee['maritalstatus'] = EDA_humanresources_employee['maritalstatus'].str.strip().str.upper()

# Criar um backup do df tratado
EDA_humanresources_employee_bkp_v2 = EDA_humanresources_employee.copy()


In [253]:
# Definir regex para validar números (exemplo: apenas dígitos, 9 caracteres)
# Acrescentei para ver se tinha um padrão, mas não tem.
regex = r'^\d{9}$'

# Verificar valores inválidos de acordo com a regra do regex anterior
invalid_nationalid = EDA_humanresources_employee[~EDA_humanresources_employee['nationalidnumber'].astype(str).str.match(regex)]
print(f"Valores inválidos em 'nationalidnumber':\n{invalid_nationalid['nationalidnumber']}")


Valores inválidos em 'nationalidnumber':
303    42487730
304    56920285
305    24756624
308    52541318
311    95958330
313    72636981
317    14417807
327     6298838
335    66073987
336    33237992
342     9659517
346    10708100
350    92096924
354     8066363
356    63179277
359    36151748
371    58791499
374     1662732
376     7201901
378    90888098
379    82638150
402    54759846
421     1300049
423    45615666
426    63761469
427    25011600
432    56772045
443    97728960
453    65848458
455    60114406
462    87268837
487    19312190
499    20244403
519    28414965
530       30845
532    60517918
548    20269531
556    58317344
569    61161660
573    90836195
Name: nationalidnumber, dtype: int64


In [254]:
# Listar colunas binárias esperadas
binary_columns = ['currentflag', 'salariedflag']

# Verificar valores únicos em colunas binárias
for col in binary_columns:
    unique_values = EDA_humanresources_employee[col].unique()
    print(f"Valores únicos em '{col}': {unique_values}") 


Valores únicos em 'currentflag': [ True]
Valores únicos em 'salariedflag': [ True False]


In [255]:
# Contar valores em 'currentflag' e 'salariedflag'
print("Distribuição de 'currentflag':")
print(EDA_humanresources_employee['currentflag'].value_counts())

print("\nDistribuição de 'salariedflag':")
print(EDA_humanresources_employee['salariedflag'].value_counts())

Distribuição de 'currentflag':
currentflag
True    290
Name: count, dtype: int64

Distribuição de 'salariedflag':
salariedflag
False    238
True      52
Name: count, dtype: int64


In [256]:
# Verificar se há funcionários ativos com `false`, já que `false` indica demitido/desligado, portanto, não deveriam ser considerados ativos
print("Funcionários ativos errados:", EDA_humanresources_employee[EDA_humanresources_employee['currentflag'] != True])

# Validar datas
print("Contratações futuras:", EDA_humanresources_employee[pd.to_datetime(EDA_humanresources_employee['hiredate'], errors='coerce') > pd.Timestamp.now()])
print("Modifieddate antes de hiredate:", EDA_humanresources_employee[pd.to_datetime(EDA_humanresources_employee['modifieddate'], errors='coerce') < pd.to_datetime(EDA_humanresources_employee['hiredate'], errors='coerce')])

Funcionários ativos errados: Empty DataFrame
Columns: [birthdate, businessentityid, currentflag, gender, hiredate, jobtitle, loginid, maritalstatus, modifieddate, nationalidnumber, organizationnode, rowguid, salariedflag, sickleavehours, vacationhours]
Index: []
Contratações futuras: Empty DataFrame
Columns: [birthdate, businessentityid, currentflag, gender, hiredate, jobtitle, loginid, maritalstatus, modifieddate, nationalidnumber, organizationnode, rowguid, salariedflag, sickleavehours, vacationhours]
Index: []
Modifieddate antes de hiredate: Empty DataFrame
Columns: [birthdate, businessentityid, currentflag, gender, hiredate, jobtitle, loginid, maritalstatus, modifieddate, nationalidnumber, organizationnode, rowguid, salariedflag, sickleavehours, vacationhours]
Index: []


In [257]:
EDA_humanresources_employee.info()

<class 'pandas.core.frame.DataFrame'>
Index: 290 entries, 290 to 579
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   birthdate         290 non-null    object
 1   businessentityid  290 non-null    int64 
 2   currentflag       290 non-null    bool  
 3   gender            290 non-null    object
 4   hiredate          290 non-null    object
 5   jobtitle          290 non-null    object
 6   loginid           290 non-null    object
 7   maritalstatus     290 non-null    object
 8   modifieddate      290 non-null    object
 9   nationalidnumber  290 non-null    int64 
 10  organizationnode  290 non-null    object
 11  rowguid           290 non-null    object
 12  salariedflag      290 non-null    bool  
 13  sickleavehours    290 non-null    int64 
 14  vacationhours     290 non-null    int64 
dtypes: bool(2), int64(4), object(9)
memory usage: 40.4+ KB


In [258]:
# Atualizar o dicionário df_processados com o df ajustado
df_processados['humanresources_employee'] = EDA_humanresources_employee

In [259]:
# Estatística descritiva para verificar se ainda há o que ser feito antes de exportar os dados ao BigQuery

# Identificar colunas numéricas para análise de outliers
numeric_columns = ['sickleavehours', 'vacationhours']

# Exibir estatísticas descritivas para entender a variação e centralidade dos dados.
print(EDA_humanresources_employee[numeric_columns].describe())

# Calcular limites para outliers (IQR - Intervalo Interquartil), para verificar valores atípicos (remoção, capping, imputação)
    # Remoção: excluir as linhas
    # Capping: substituir os valores fora dos limites pelo valor do limite inferior ou superior
    # Imputação: substituir os valores outliers por medidas centrais, como a média ou mediana, se isso fizer sentido

for col in numeric_columns:
    q1 = EDA_humanresources_employee[col].quantile(0.25)
    q3 = EDA_humanresources_employee[col].quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    
    # Exibir os limites
    print(f"\nColuna: {col}")
    print(f"Limite inferior: {lower_bound}, Limite superior: {upper_bound}")
    
    # Filtrar outliers
    outliers = EDA_humanresources_employee[(EDA_humanresources_employee[col] < lower_bound) | (EDA_humanresources_employee[col] > upper_bound)]
    print(f"Outliers detectados ({len(outliers)}):")
    print(outliers[[col]])


       sickleavehours  vacationhours
count          290.00         290.00
mean            45.31          50.61
std             14.54          28.79
min             20.00           0.00
25%             33.00          26.25
50%             46.00          51.00
75%             58.00          75.00
max             80.00          99.00

Coluna: sickleavehours
Limite inferior: -4.5, Limite superior: 95.5
Outliers detectados (0):
Empty DataFrame
Columns: [sickleavehours]
Index: []

Coluna: vacationhours
Limite inferior: -46.875, Limite superior: 148.125
Outliers detectados (0):
Empty DataFrame
Columns: [vacationhours]
Index: []


In [260]:
# Garantir que apenas tabelas únicas sejam exportadas
unique_df_processados = {k: v for k, v in df_processados.items()}

# Exportar tabelas para o BigQuery
for table_name, df_cleaned in unique_df_processados.items():
    # Remover o prefixo 'stg_' para exibição e exportação
    clean_table_name = table_name.replace("stg_", "")

    # Nome da tabela no BigQuery sem 'stg_'
    output_table = f"{output_dataset}.{clean_table_name}"

    schema = [
        bigquery.SchemaField("birthdate", "DATE"),
        bigquery.SchemaField("businessentityid", "INTEGER"),
        bigquery.SchemaField("currentflag", "BOOLEAN"),
        bigquery.SchemaField("gender", "STRING"),
        bigquery.SchemaField("hiredate", "DATE"),
        bigquery.SchemaField("jobtitle", "STRING"),
        bigquery.SchemaField("loginid", "STRING"),
        bigquery.SchemaField("maritalstatus", "STRING"),
        bigquery.SchemaField("modifieddate", "DATE"),
        bigquery.SchemaField("nationalidnumber", "INTEGER"),
        bigquery.SchemaField("organizationnode", "STRING"),
        bigquery.SchemaField("rowguid", "STRING"),
        bigquery.SchemaField("salariedflag", "BOOLEAN"),
        bigquery.SchemaField("sickleavehours", "INTEGER"),
        bigquery.SchemaField("vacationhours", "INTEGER"),
    ]
    filtered_schema = [field for field in schema if field.name in df_cleaned.columns]

    job_config = bigquery.LoadJobConfig(
        source_format=bigquery.SourceFormat.CSV,
        skip_leading_rows=0,
        write_disposition="WRITE_TRUNCATE",
        schema=filtered_schema,
    )
    job = client.load_table_from_dataframe(df_cleaned, output_table, job_config=job_config)
    job.result()

    print(f"Tabela {clean_table_name} exportada com sucesso para {output_table}.")


Tabela humanresources_employee exportada com sucesso para desafioadventureworks-446600.raw_data_cleaned.humanresources_employee.
